<a href="https://colab.research.google.com/github/esmanp/Geri-Bildir/blob/main/Model_E%C4%9Fitim_S%C3%BCreci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Text Classification/ Sentiment için ML Modeller

In [ ]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet'}
train = pd.read_parquet("hf://datasets/Gorengoz/tr-customerreview/" + splits["train"])

In [ ]:
train

,description,durum
0,2 ürün istedim tek ürün gönderdiler. Hediye al...,olumsuz
1,Ürünü günlerce bekledim. Kargoya verildi ve hi...,olumsuz
2,gayet kaliteli başarılı bir ürün,olumlu
3,İphone kullanıcısıyım power bank ve kulaklıkla...,olumlu
4,vga kablom çalışır durumda olmasına rağmen bağ...,olumlu
...,...,...
2628,Ürün çimleri bicmiyor motor calismasinda skint...,olumsuz
2629,"Sağlam, hızlı aktarılıyor, çabuk geldi.",olumlu
2630,10'a yakın farklı kablo ile denedim. Bir çok k...,nötr
2631,Satın almayı düşünen kişiler için görüşlerimi ...,olumlu


In [ ]:
import pandas as pd

splits = {'train': 'data/train-00000-of-00001.parquet', 'validation': 'data/validation-00000-of-00001.parquet'}
test= pd.read_parquet("hf://datasets/Gorengoz/tr-customerreview/" + splits["validation"])
test

,description,durum
0,Aşağı doğru eğimi çok yeterli değil. Hafif bir...,nötr
1,3. Kademe çalışmıyor plastiği çok kötü her an ...,olumsuz
2,Güzel halı fakat ben özellikle püskül olmasın ...,olumsuz
3,Kullanışlı,olumlu
4,Güzel,olumlu
...,...,...
288,üzülerek bir yıldız veriyorum. çok güzel çok d...,olumsuz
289,gayet kaliteli biraz plastik kokuyor havalandı...,olumlu
290,Xl sipariş verdim s benden gibi küçük,olumsuz
291,Aktarım hızlı değil!,olumlu


In [ ]:
frames = [train, test]

df= pd.concat(frames)
df

,description,durum
0,2 ürün istedim tek ürün gönderdiler. Hediye al...,olumsuz
1,Ürünü günlerce bekledim. Kargoya verildi ve hi...,olumsuz
2,gayet kaliteli başarılı bir ürün,olumlu
3,İphone kullanıcısıyım power bank ve kulaklıkla...,olumlu
4,vga kablom çalışır durumda olmasına rağmen bağ...,olumlu
...,...,...
288,üzülerek bir yıldız veriyorum. çok güzel çok d...,olumsuz
289,gayet kaliteli biraz plastik kokuyor havalandı...,olumlu
290,Xl sipariş verdim s benden gibi küçük,olumsuz
291,Aktarım hızlı değil!,olumlu


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt


      Unzipping tokenizers/punkt.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:

!pip install TurkishStemmer

In [ ]:
import re
import string
from TurkishStemmer import TurkishStemmer
df["description"]= df["description"].apply(lambda x : x.lower()) # küçük harfe çevirme
df["description"]= df["description"].apply(lambda x : re.sub(r'\d', '', x)) # sayıları kaldırma
df["description"]= df["description"].apply(lambda x : ''.join(d for d in x if d not in string.punctuation)) # noktalama işaretleri kaldırma
df["description"]= df["description"].apply(lambda x : TurkishStemmer().stem(x)) # cümleyi kelimelere ayırma
df["description"]= df["description"].apply(lambda x : word_tokenize(x)) # cümleyi köklerine ayırma

In [ ]:
df.description = df.description.apply(' '.join)


In [ ]:
import pandas as pd
import numpy as np

import string
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
cv = CountVectorizer(max_features = 3000)
X = cv.fit_transform(df.description).toarray()
y = df.durum

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#Multinomial Naive Bayes
mnb = MultinomialNB()
mnb.fit(X_train,y_train)
y_pred = mnb.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


#SVM Classifier
sgd = SVC()
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))



0.8333333333333334
[[ 19  55  27]
 [ 10 489  23]
 [ 17  29 297]]
              precision    recall  f1-score   support

        nötr       0.41      0.19      0.26       101
      olumlu       0.85      0.94      0.89       522
     olumsuz       0.86      0.87      0.86       343

    accuracy                           0.83       966
   macro avg       0.71      0.66      0.67       966
weighted avg       0.81      0.83      0.82       966

0.8002070393374742
[[  7  68  26]
 [  0 504  18]
 [  0  81 262]]
              precision    recall  f1-score   support

        nötr       1.00      0.07      0.13       101
      olumlu       0.77      0.97      0.86       522
     olumsuz       0.86      0.76      0.81       343

    accuracy                           0.80       966
   macro avg       0.88      0.60      0.60       966
weighted avg       0.83      0.80      0.76       966



### Text Classifacation Modellleri Kıyaslaması

In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 478.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.9 MB/s eta 0:00:00
 

In [ ]:
import numpy as np
import pandas as pd


#!pip install simpletransformers
from simpletransformers.classification import ClassificationModel

In [ ]:
dt= pd.read_excel("/content/run_results.xlsx")
dt.head()

,description,durum
0,Çoookk memnunum iyi ki almışım. Ürün dayanıklı...,olumlu
1,"Ürünü Prime kampanyasından 189,00₺ gibi bir ra...",olumlu
2,15 inc bir Macbook Pro ile kullanıyorum. Daha ...,olumlu
3,hani metali tutar da soğukluğundan kaliteyi hi...,olumlu
4,Hafif ve gayet kullanışlı. Şık kolay katlanır ...,olumlu


In [ ]:
dt["labels"] = pd.factorize(dt.durum)[0]

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(dt, test_size=0.2, random_state=42)


train=train[["description","labels"]]
test=test[["description","labels"]]


#for bert text = string      label = int
train["description"]=train["description"].apply(lambda r: str(r))
train['labels']=train['labels'].astype(int)

In [ ]:
# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    'dbmdz/bert-base-turkish-uncased',
    num_labels=3
)

# Train the model
model.train_model(train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/293 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
predictions = model_outputs.argmax(axis=1)

actuals = test.labels.values

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

0.8327645051194539

In [ ]:
# Create a ClassificationModel
model_args={"overwrite_output_dir": True}
model = ClassificationModel(
    'electra',
    'dbmdz/electra-base-turkish-mc4-cased-discriminator',
    num_labels=3,
    args=model_args
)

# Train the model
model.train_model(train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test)
predictions = model_outputs.argmax(axis=1)

actuals = test.labels.values

from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at dbmdz/electra-base-turkish-mc4-cased-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/4 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 1 of 1:   0%|          | 0/293 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

0.8276450511945392

In [ ]:
# Create a ClassificationModel
model_args={"overwrite_output_dir": True}
model = ClassificationModel(
    'distilbert',
    'dbmdz/distilbert-base-turkish-cased',
    num_labels=3,
    args=model_args
)

# Train the model
model.train_model(train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test)
predictions = model_outputs.argmax(axis=1)

actuals = test.labels.values

from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

config.json:   0%|          | 0.00/410 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/distilbert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

...

Running Epoch 1 of 1:   0%|          | 0/293 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

0.7935153583617748

In [ ]:

# Create a ClassificationModel
model_args={"overwrite_output_dir": True}
model = ClassificationModel(
    'roberta',
    'FacebookAI/xlm-roberta-base',
    num_labels=3,
    args=model_args
)

# Train the model
model.train_model(train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test)
predictions = model_outputs.argmax(axis=1)

actuals = test.labels.values

from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

Running Epoch 1 of 1:   0%|          | 0/293 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

0.7764505119453925

In [ ]:
# Create a ClassificationModel
model_args={"overwrite_output_dir": True}
model = ClassificationModel(
    'xlnet',
    'xlnet/xlnet-base-cased',
    num_labels=3,
    args=model_args
)

# Train the model
model.train_model(train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test)
predictions = model_outputs.argmax(axis=1)

actuals = test.labels.values

from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet/xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

...

Running Epoch 1 of 1:   0%|          | 0/293 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

0.7491467576791809



---



#### Text Classification Modelleri (LLM)

##### Gorengoz/bert-turkish-sentiment-analysis-cased

In [ ]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 97.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 113.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 28.3 MB/s eta 0:00:00
 

In [ ]:
import numpy as np
import pandas as pd


#!pip install simpletransformers
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
dt= pd.read_excel("/content/run_results.xlsx")
dt.head()

,description,durum
0,Çoookk memnunum iyi ki almışım. Ürün dayanıklı...,olumlu
1,"Ürünü Prime kampanyasından 189,00₺ gibi bir ra...",olumlu
2,15 inc bir Macbook Pro ile kullanıyorum. Daha ...,olumlu
3,hani metali tutar da soğukluğundan kaliteyi hi...,olumlu
4,Hafif ve gayet kullanışlı. Şık kolay katlanır ...,olumlu


In [ ]:
dt["labels"] = pd.factorize(dt.durum)[0]

In [ ]:
dt

,description,durum,labels
0,Çoookk memnunum iyi ki almışım. Ürün dayanıklı...,olumlu,0
1,"Ürünü Prime kampanyasından 189,00₺ gibi bir ra...",olumlu,0
2,15 inc bir Macbook Pro ile kullanıyorum. Daha ...,olumlu,0
3,hani metali tutar da soğukluğundan kaliteyi hi...,olumlu,0
4,Hafif ve gayet kullanışlı. Şık kolay katlanır ...,olumlu,0
...,...,...,...
2921,Almayin. Pervane carpiyor ayrica plastikten ta...,olumsuz,2
2922,ürün hatalı gönderildi. arka kafes yerine iki ...,olumsuz,2
2923,Abartisisiz oyuncak gibi kendine faydası yok s...,olumsuz,2
2924,aldığım ürünün ara borusu yok iadeetmek istiyo...,olumsuz,2


In [ ]:
from sklearn.model_selection import train_test_split
import torch,sklearn

train, test = train_test_split(dt, test_size=0.2, random_state=42)


train=train[["description","labels"]]
test=test[["description","labels"]]


#for bert text = string      label = int
train["description"]=train["description"].apply(lambda r: str(r))
train['labels']=train['labels'].astype(int)

In [ ]:
model_args = {
     "num_train_epochs":3, "overwrite_output_dir":True}

# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    'dbmdz/bert-base-turkish-128k-cased',
    num_labels=3,
    args=model_args,
    weight=[0.1, 0.25,0.65]
)


# Train the model
model.train_model(train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/386 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

...

Running Epoch 2 of 3:   0%|          | 0/293 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/293 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
predictions = model_outputs.argmax(axis=1)

actuals = test.labels.values

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

0.8464163822525598

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline

tokenizer = AutoTokenizer.from_pretrained("Gorengoz/bert-turkish-sentiment-analysis-cased")
model = AutoModelForSequenceClassification.from_pretrained("Gorengoz/bert-turkish-sentiment-analysis-cased")

nlp=pipeline("text-classification", model=model, tokenizer=tokenizer)

code_to_label={

'LABEL_0': 'olumlu ',

'LABEL_1': 'nötr ',

'LABEL_2': 'olumsuz' }

code_to_label[nlp("Ürün berbat, paranıza yazık olur.")[0]['label']]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/737M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


'olumsuz'

##### Gorengoz/bert-turkish-sentiment-analysis-winvoker

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
     

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load the dataset
dataset = load_dataset('winvoker/turkish-sentiment-analysis-dataset')

# Convert to DataFrame
df = dataset['train'].to_pandas()

# Display the DataFrame to understand its structure
print("Original DataFrame:")
print(df.head())

# Define the class column and number of samples
class_column = 'label'  # Adjust this if the class column has a different name
n_samples = 5000

# Group by the class column and sample from each group
sampled_df = df.groupby(class_column).apply(lambda x: x.sample(n=min(n_samples, len(x)), replace=False)).reset_index(drop=True)

# Display the sampled DataFrame
print("\nSampled DataFrame:")
print(sampled_df)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/440679 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/48965 [00:00<?, ? examples/s]

Original DataFrame:
                                                text     label         dataset
0  ürünü hepsiburadadan alalı 3 hafta oldu. orjin...  Positive  urun_yorumlari
1  ürünlerden çok memnunum, kesinlikle herkese ta...  Positive  urun_yorumlari
2      hızlı kargo, temiz alışveriş.teşekkür ederim.  Positive  urun_yorumlari
3               Çünkü aranan tapınak bu bölgededir .      Notr            wiki
4  bu telefonu başlıca alma nedenlerim ise elimde...  Positive  urun_yorumlari

Sampled DataFrame:
                                                    text     label  \
0       filmi izlerken bu filmi yapanların bize hakar...  Negative   
1      koku artık eskisi kadar kalıcı değil önceden s...  Negative   
2      Elime geçen ürün başta çok iyi görünüyordu 1 h...  Negative   
3                  Ergen filmi bile değil, zaman kaybı.   Negative   
4                                        kafayı yedi lan  Negative   
...                                                  ...       ...

In [ ]:
sampled_df = sampled_df.drop(columns=['dataset'])

In [ ]:
sampled_df = sampled_df.rename(columns={'text': 'description', 'label': 'durum'})

In [ ]:
df = sampled_df
df['durum'] = df['durum'].replace({
    'Positive': 'olumlu',
    'Negative': 'olumsuz',
    'Notr': 'nötr'
})

In [ ]:
df

,description,durum
0,filmi izlerken bu filmi yapanların bize hakar...,olumsuz
1,koku artık eskisi kadar kalıcı değil önceden s...,olumsuz
2,Elime geçen ürün başta çok iyi görünüyordu 1 h...,olumsuz
3,"Ergen filmi bile değil, zaman kaybı.",olumsuz
4,kafayı yedi lan,olumsuz
...,...,...
14995,siyahları soldurmadan parlak bir bicimde yıkıy...,olumlu
14996,12 sene önce aldığım braun portakal sıkacağı h...,olumlu
14997,1 kerede inanılmaz fön çekiyor.saçlarım uzun o...,olumlu
14998,merhaba siparişten bir kaç gün sonra ulaştı gü...,olumlu


In [ ]:
dataset2 = load_dataset('Gorengoz/tr-customerreview')
df2 = dataset2['train'].to_pandas()

Generating train split:   0%|          | 0/2633 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/293 [00:00<?, ? examples/s]

In [ ]:
merged_df = pd.concat([df, df2], ignore_index=True)

In [ ]:
df = merged_df
df

,description,durum
0,filmi izlerken bu filmi yapanların bize hakar...,olumsuz
1,koku artık eskisi kadar kalıcı değil önceden s...,olumsuz
2,Elime geçen ürün başta çok iyi görünüyordu 1 h...,olumsuz
3,"Ergen filmi bile değil, zaman kaybı.",olumsuz
4,kafayı yedi lan,olumsuz
...,...,...
17628,Ürün çimleri bicmiyor motor calismasinda skint...,olumsuz
17629,"Sağlam, hızlı aktarılıyor, çabuk geldi.",olumlu
17630,10'a yakın farklı kablo ile denedim. Bir çok k...,nötr
17631,Satın almayı düşünen kişiler için görüşlerimi ...,olumlu


In [ ]:
df['durum'].value_counts()

,count
durum,
olumlu,6448
olumsuz,5923
nötr,5262


In [ ]:
dt = df
dt["labels"] = pd.factorize(dt.durum)[0]
dt

,description,durum,labels
0,filmi izlerken bu filmi yapanların bize hakar...,olumsuz,0
1,koku artık eskisi kadar kalıcı değil önceden s...,olumsuz,0
2,Elime geçen ürün başta çok iyi görünüyordu 1 h...,olumsuz,0
3,"Ergen filmi bile değil, zaman kaybı.",olumsuz,0
4,kafayı yedi lan,olumsuz,0
...,...,...,...
17628,Ürün çimleri bicmiyor motor calismasinda skint...,olumsuz,0
17629,"Sağlam, hızlı aktarılıyor, çabuk geldi.",olumlu,2
17630,10'a yakın farklı kablo ile denedim. Bir çok k...,nötr,1
17631,Satın almayı düşünen kişiler için görüşlerimi ...,olumlu,2


In [ ]:
from sklearn.model_selection import train_test_split
import torch,sklearn

train, test = train_test_split(dt, test_size=0.2, random_state=42)


train=train[["description","labels"]]
test=test[["description","labels"]]


#for bert text = string      label = int
train["description"]=train["description"].apply(lambda r: str(r))
train['labels']=train['labels'].astype(int)

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
model_args = {
     "num_train_epochs":3, "overwrite_output_dir":True}

# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    'dbmdz/bert-base-turkish-128k-cased',
    num_labels=3,
    args=model_args
)


# Train the model
model.train_model(train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test)

config.json:   0%|          | 0.00/386 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/740M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

...

Running Epoch 2 of 3:   0%|          | 0/1764 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/1764 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/7 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/36 [00:00<?, ?it/s]

In [ ]:
predictions = model_outputs.argmax(axis=1)

actuals = test.labels.values

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

0.933654664020414

In [ ]:
pip install huggingface_hub

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [ ]:
model_path = '/content/outputs'
repo_name = 'Gorengoz/bert-turkish-sentiment-analysis-winvoker'

In [ ]:
from transformers import AutoModel, AutoTokenizer
model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/737M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Gorengoz/bert-turkish-sentiment-analysis-winvoker/commit/5bd697df3a7b9e0f46219e84f2658fb77b1dba71', commit_message='Upload tokenizer', commit_description='', oid='5bd697df3a7b9e0f46219e84f2658fb77b1dba71', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="Gorengoz/bert-turkish-sentiment-analysis-winvoker")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Gorengoz/bert-turkish-sentiment-analysis-winvoker and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline

tokenizer = AutoTokenizer.from_pretrained("Gorengoz/bert-turkish-sentiment-analysis-winvoker")
model = AutoModelForSequenceClassification.from_pretrained("Gorengoz/bert-turkish-sentiment-analysis-winvoker")

nlp=pipeline("text-classification", model=model, tokenizer=tokenizer)

code_to_label={

'LABEL_0': 'olumsuz ',

'LABEL_1': 'nötr ',

'LABEL_2': 'olumlu' }

code_to_label[nlp("Ürün berbat, paranıza yazık olur.")[0]['label']]


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/737M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Gorengoz/bert-turkish-sentiment-analysis-winvoker and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


'olumsuz '

## NER Modelleri

####  Gorengoz/bert-based-Turkish-NER  & Gorengoz/bert-based-Turkish-NER-wikiann

In [ ]:
from simpletransformers.ner.ner_model import NERModel

custom_labels = ["O","B-LOC","I-LOC","B-ORG","I-ORG","B-PER","I-PER"]
model_args = {
    "train_batch_size": 32,
    "eval_batch_size": 32,
    "num_train_epochs": 3,
    "seed":1,
    "save_steps": 625,
    "overwrite_output_dir": True,
    "output_dir": "/content/Model"
}

# Create a NERModel
model = NERModel('bert', 'dbmdz/bert-base-turkish-cased',labels=custom_labels,args=model_args )

# Train the model
model.train_model('/content/train.txt')

# Evaluate the model
result, model_outputs, predictions = model.eval_model('/content/test.txt')

# Check predictions
print(predictions[:5])

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Running Epoch 1 of 3:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 2 of 3:   0%|          | 0/625 [00:00<?, ?it/s]

Running Epoch 3 of 3:   0%|          | 0/625 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/313 [00:00<?, ?it/s]

[['B-ORG', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O'], ['O', 'B-LOC', 'I-LOC', 'I-LOC'], ['O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'B-LOC', 'O', 'O', 'O']]


In [ ]:
import torch
import torch.nn as nn

FILE = "model.pth"

torch.save(model, FILE)


In [ ]:
model = torch.load(FILE)
model.eval()

In [ ]:
predictions, raw_outputs = model.predict(["Fiber 100mb SuperOnline kullanıcısıyım yaklaşık 2 haftadır Twitch Kick_Turkey gibi canlı yayın platformlarında 360p yayın izlerken donmalar yaşıyoruz. Başka hiç bir operatörler bu sorunu yaşamazken ben parasını verip alamadığım hizmeti neden ödeyeyim ? Turkcell"])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
predictions

[[{'Fiber': 'O'},
  {'100mb': 'O'},
  {'SuperOnline': 'B-ORG'},
  {'kullanıcısıyım': 'O'},
  {'yaklaşık': 'O'},
  {'2': 'O'},
  {'haftadır': 'O'},
  {'Twitch': 'B-ORG'},
  {'Kick_Turkey': 'B-ORG'},
  {'gibi': 'O'},
  {'canlı': 'O'},
  {'yayın': 'O'},
  {'platformlarında': 'O'},
  {'360p': 'O'},
  {'yayın': 'O'},
  {'izlerken': 'O'},
  {'donmalar': 'O'},
  {'yaşıyoruz.': 'O'},
  {'Başka': 'O'},
  {'hiç': 'O'},
  {'bir': 'O'},
  {'operatörler': 'O'},
  {'bu': 'O'},
  {'sorunu': 'O'},
  {'yaşamazken': 'O'},
  {'ben': 'O'},
  {'parasını': 'O'},
  {'verip': 'O'},
  {'alamadığım': 'O'},
  {'hizmeti': 'O'},
  {'neden': 'O'},
  {'ödeyeyim': 'O'},
  {'?': 'O'},
  {'Turkcell': 'B-ORG'}]]

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("token-classification", model="Gorengoz/bert-based-Turkish-NER")

config.json:   0%|          | 0.00/990 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
pipe("Fiber 100mb SuperOnline kullanıcısıyım yaklaşık 2 haftadır Twitch Kick_Turkey gibi canlı yayın platformlarında 360p yayın izlerken donmalar yaşıyoruz. Başka hiç bir operatörler bu sorunu yaşamazken ben parasını verip alamadığım hizmeti neden ödeyeyim ? Turkcell",aggregation_strategy = "simple")

[{'entity_group': 'ORGANIZATION',
  'score': 0.77373785,
  'word': 'SuperOnline',
  'start': 12,
  'end': 23},
 {'entity_group': 'ORGANIZATION',
  'score': 0.6571868,
  'word': 'Twit',
  'start': 59,
  'end': 63},
 {'entity_group': 'ORGANIZATION',
  'score': 0.99595636,
  'word': 'Turkcell',
  'start': 253,
  'end': 261}]

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe2 = pipeline("token-classification", model="Gorengoz/bert-based-Turkish-NER-wikiann")

config.json:   0%|          | 0.00/922 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
pipe2(" Entity X'in müşteri hizmetleri hızlı ve etkili, Entity Y'nin ürün kalitesi çok kötü.",aggregation_strategy = "simple")

[{'entity_group': 'ORG',
  'score': 0.91663206,
  'word': 'Entity X',
  'start': 1,
  'end': 9},
 {'entity_group': 'ORG',
  'score': 0.9230066,
  'word': 'Entity Y',
  'start': 49,
  'end': 57}]